In [94]:
import numpy as np 
import pandas as pd
from haversine import haversine_vector, Unit
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union

State="NC"
GeneratorDataPath="./EIAData/february_generator2023.xlsx"
HazusDataPath="./HazusData/Hazus_NC_Hurricane.shp"


Vintages=np.arange(2022,1900,-2) #[2022 to 2020), [2020 to 2018) Needs to start from earlier date to later date. Needs equal spacing

AggregationDistance=16#[km], Distance technologies need to be in order to get aggregated with each other 
#Only technologies of the same vintage period, and type will be aggregated

# Code to Convert Excel Table to Sql data

## Generation 


In [95]:
##Generation data from: Preliminary Monthly Electric Generator Inventory (based on Form EIA-860M as a supplement to Form EIA-860)

#Read File
df = pd.read_excel(GeneratorDataPath, sheet_name ='Operating',skiprows=2)
InState=df["Plant State"]==State #Filter for NC
df=df[InState]

In [96]:
#   Some plants may have multiple generators commissioned at different times
#   we are aggregating the capacity of these generators per year

#   We can have multiple vintages (years) of the same plant. As generators may be comissioned at different times
df_AggregateGen = pd.DataFrame(columns=["PlantID", "SourceCode", "MoverCode", "OperatingYear","NameplateCapacity(MW)","NetSummerCapacity(MW)","NetWinterCapacity(MW)",
 "NameplateEnergyCapacity(MWh)", "Latitude","Longitude","Technology"])

for ID in df["Plant ID"].unique().astype(int):
    #Same Plant data
    SameIds   = df["Plant ID"]==ID
    df_tmp1    = df[SameIds]

    SourceCode=df_tmp1["Energy Source Code"]
    #Same Energy Source Code data
    for UniqueSC in SourceCode.unique():
        SameIds    = df_tmp1["Energy Source Code"]==UniqueSC
        df_tmp2    = df_tmp1[SameIds]

        MoverCode=df_tmp2["Prime Mover Code"]
        #Same Mover Code data
        for UniqueMC in MoverCode.unique():
            SameIds    = df_tmp2["Prime Mover Code"]==UniqueMC
            df_tmp3    = df_tmp2[SameIds]

            OperatingYear=df_tmp3["Operating Year"]
            OperatingYear=OperatingYear.sort_values(ascending=False)
            #Same Year Code data
            for UniqueY in OperatingYear.unique():
                SameIds    = df_tmp3["Operating Year"]==UniqueY
                df_tmp4    = df_tmp3[SameIds]

                NameplateEnergyCapacity=np.sum(df_tmp4["Nameplate Energy Capacity (MWh)"])
                NameplateCapacity=np.sum(df_tmp4["Nameplate Capacity (MW)"])
                NetSummerCapacity=np.sum(df_tmp4["Net Summer Capacity (MW)"])
                NetWinterCapacity=np.sum(df_tmp4["Net Winter Capacity (MW)"])

                Latitude  = df_tmp4["Latitude"].iloc[0]
                Longitude = df_tmp4["Longitude"].iloc[0]
                Technology= df_tmp4["Technology"].iloc[0]

                NewPlantID = UniqueSC + "_" + UniqueMC + "_" + str(ID) 

                Data=[[NewPlantID, UniqueSC, UniqueMC, UniqueY, NameplateCapacity, NetSummerCapacity, NetWinterCapacity,NameplateEnergyCapacity,
                Latitude,Longitude,Technology]]

                df_AggregateGen=pd.concat([df_AggregateGen,pd.DataFrame(Data,columns=df_AggregateGen.columns)],ignore_index=True)

#Delete technologies that are after the maximum vintage period
df_AggregateGen=df_AggregateGen[df_AggregateGen["OperatingYear"]<=np.max(Vintages)]

#Check for missing data
if sum(df_AggregateGen[df_AggregateGen["MoverCode"]=="BA"]["NameplateEnergyCapacity(MWh)"]==' '):
    EliminateIdx=(df_AggregateGen["MoverCode"]=="BA") * (df_AggregateGen["NameplateEnergyCapacity(MWh)"]==' ')
    EliminateId=df_AggregateGen[EliminateIdx]["PlantID"]
    print("Check Battery Data, some elements do not have NameplateEnergyCapacity(MWh) and were removed:\n Plant ID:")
    [print(EliminateId.iloc[i]) for i in range(EliminateId.shape[0])]

    df_AggregateGen=df_AggregateGen[~EliminateIdx]

df_AggregateGen = df_AggregateGen.reset_index(drop=True)

Check Battery Data, some elements do not have NameplateEnergyCapacity(MWh) and were removed:
 Plant ID:
MWH_BA_64640


### Aggregate per radius 

In [257]:
#Grup generators too close to each other, in the same vintage bracket
#and of the same type and fuel "SourceCode", "MoverCode"

#   We can have multiple vintages (years) of the same plant. As generators may be comissioned at different times
df_AggregateGen2 = pd.DataFrame(columns=["PlantID", "SourceCode", "MoverCode", "Vintage","OperatingYear","NameplateCapacity(MW)","NetSummerCapacity(MW)","NetWinterCapacity(MW)",
 "NameplateEnergyCapacity(MWh)", "Latitude","Longitude","Technology"])

#Compute distance matrix for each element on df_Aggregation
Latitude=np.reshape(df_AggregateGen["Latitude"].to_numpy(),(df_AggregateGen["Latitude"].shape[0],1))
Latitude=np.tile(Latitude,(1,Latitude.shape[0]))

Longitude=np.reshape(df_AggregateGen["Longitude"].to_numpy(),(df_AggregateGen["Longitude"].shape[0],1))
Longitude=np.tile(Longitude,(1,Longitude.shape[0]))

X_LatLong=np.array([np.reshape(Latitude,(-1)),np.reshape(Longitude,(-1))]).T
Y_LatLong=np.array([np.reshape(Latitude.T,(-1)),np.reshape(Longitude.T,(-1))]).T

Distances=haversine_vector(X_LatLong,Y_LatLong, Unit.KILOMETERS)
Distances=np.reshape(Distances,(Latitude.shape[0],Longitude.shape[0])) #Distances between any pair of available generators

ID=0
while df_AggregateGen.shape[0]!=0:
    ID=ID+1
    RefGenerator=df_AggregateGen.iloc[0]
    IdxInDistance=Distances[0,:]<=AggregationDistance
    IdxSameSource=df_AggregateGen["SourceCode"]==RefGenerator["SourceCode"]
    IdxSameMover=df_AggregateGen["MoverCode"]==RefGenerator["MoverCode"]

    #Vintage Bracket of the current generator
    if np.min(np.abs(Vintages-RefGenerator["OperatingYear"]))==0:
        VintageMax=RefGenerator["OperatingYear"]
        VintageMin=RefGenerator["OperatingYear"]-(Vintages[0]-Vintages[1])

    else:
        IdxVintage=np.argsort(np.abs(Vintages-RefGenerator["OperatingYear"]))[0:2] 
        VintageMax=np.max(Vintages[IdxVintage])
        VintageMin=np.min(Vintages[IdxVintage])

    IdxSameVintage=(df_AggregateGen["OperatingYear"]>VintageMin)*(df_AggregateGen["OperatingYear"]<=VintageMax)

    IdxToAggregate=IdxSameVintage*IdxSameMover*IdxSameSource*IdxInDistance

    df_tmp=df_AggregateGen[IdxToAggregate]

    NameplateCapacity=np.sum(df_tmp["NameplateCapacity(MW)"])
    NetSummerCapacity=np.sum(df_tmp["NetSummerCapacity(MW)"])
    NetWinterCapacity=np.sum(df_tmp["NetWinterCapacity(MW)"])
    NameplateEnergyCapacity=np.sum(df_tmp["NameplateEnergyCapacity(MWh)"])

    #Lat Long, vintage and year of operation are estimated based on a weighted average of the Nplate Capacity
    NPC=df_tmp["NameplateCapacity(MW)"]
    Latitude=np.sum(NPC*df_tmp["Latitude"])/NameplateCapacity
    Longitude=np.sum(NPC*df_tmp["Longitude"])/NameplateCapacity
    OperatingYear=np.sum(NPC*df_tmp["OperatingYear"])/NameplateCapacity
    VintageYear=Vintages[np.argmin(np.abs(Vintages-OperatingYear))]

    NewPlantID = UniqueSC + "_" + UniqueMC + "_" + str(ID) 
    Technology= RefGenerator["Technology"]

    Data=[[NewPlantID, RefGenerator["SourceCode"], RefGenerator["MoverCode"], VintageYear, OperatingYear,
    NameplateCapacity, NetSummerCapacity, NetWinterCapacity, NameplateEnergyCapacity,
    Latitude, Longitude, Technology]]

    df_AggregateGen2=pd.concat([df_AggregateGen2,pd.DataFrame(Data,columns=df_AggregateGen2.columns)],ignore_index=True)

    #delete generators that were aggregated
    df_AggregateGen=df_AggregateGen[~IdxToAggregate]
    Distances=Distances[~IdxToAggregate,:]
    Distances=Distances[:,~IdxToAggregate]


### Aggregate per region

In [97]:
#Do not run Aggregate per radius if you want to aggregate using .shp files
shapefile = gpd.read_file(HazusDataPath)
Hazus1000=shapefile.f1000yr

Region1Idx=Hazus1000>=130 # Cat 4 hurricane and above 130mph
Region2Idx=(Hazus1000>=111)*(Hazus1000<130) # Cat 3 hurricane 111-129mph
Region3Idx=Hazus1000<111 # Cat 1-2 hurricane and bellow 110mph

Region1Data=shapefile[Region1Idx] 
Region2Data=shapefile[Region2Idx]    
Region3Data=shapefile[Region3Idx]     

new_polyR1 = unary_union([Region1Data.iloc[j]["geometry"] for j in range(Region1Data.shape[0])])
new_polyR2 = unary_union([Region2Data.iloc[j]["geometry"] for j in range(Region2Data.shape[0])])
new_polyR3 = unary_union([Region3Data.iloc[j]["geometry"] for j in range(Region3Data.shape[0])])

df_AggregateGen["Region"]=''
#Assign region for each generation 

for i in range(df_AggregateGen.shape[0]):
    Latitude=df_AggregateGen.iloc[i]["Latitude"]
    Longitude=df_AggregateGen.iloc[i]["Longitude"]

    if new_polyR1.contains(Point(Longitude,Latitude)):
        df_AggregateGen.at[i,"Region"]='R1'

    elif new_polyR2.contains(Point(Longitude,Latitude)):
        df_AggregateGen.at[i,"Region"]='R2'
    
    elif new_polyR3.contains(Point(Longitude,Latitude)):
        df_AggregateGen.at[i,"Region"]='R3'

    else:
        print("Error")
        break

if sum(df_AggregateGen.Region=='')==0:
    print("All generators were properly mapped")
else:
    print("Some generators were no properly mapped to the Hazus data\
    check Hazus and the location of generators")

All generators were properly mapped


In [108]:
df_AggregateGen3 = pd.DataFrame(columns=["PlantID", "SourceCode", "MoverCode", "Vintage","NameplateCapacity(MW)","NetSummerCapacity(MW)","NetWinterCapacity(MW)",
 "NameplateEnergyCapacity(MWh)","Technology","Region"])


for Region in ["R1","R2","R3"]:

    df_tmp1=df_AggregateGen[df_AggregateGen["Region"]==Region]

    SourceCode=df_tmp1["SourceCode"]
    #Same Energy Source Code data
    for UniqueSC in SourceCode.unique():
        SameIds    = df_tmp1["SourceCode"]==UniqueSC
        df_tmp2    = df_tmp1[SameIds]

        MoverCode=df_tmp2["MoverCode"]
        #Same Mover Code data
        for UniqueMC in MoverCode.unique():
            SameIds    = df_tmp2["MoverCode"]==UniqueMC
            df_tmp3    = df_tmp2[SameIds]
            
            OperatingYear=df_tmp3["OperatingYear"]
            for i in range(len(Vintages[0:-1])):

                IdxsIn=(OperatingYear<=Vintages[i]+(Vintages[0]-Vintages[1])/2) * (OperatingYear>Vintages[i]-(Vintages[0]-Vintages[1])/2)
                if sum(IdxsIn)!=0:
                    df_tmp4    = df_tmp3[IdxsIn]

                    NameplateCapacity=np.sum(df_tmp4["NameplateCapacity(MW)"])
                    NetSummerCapacity=np.sum(df_tmp4["NetSummerCapacity(MW)"])
                    NetWinterCapacity=np.sum(df_tmp4["NetWinterCapacity(MW)"])
                    NameplateEnergyCapacity=np.sum(df_tmp4["NameplateEnergyCapacity(MWh)"])
                    
                    NewPlantID = Region + "_" + UniqueSC + "_" + UniqueMC
                    Technology= df_tmp4["Technology"].iloc[0]

                    Data=[[NewPlantID, UniqueSC, UniqueMC, Vintages[i] ,
                    NameplateCapacity, NetSummerCapacity, NetWinterCapacity, NameplateEnergyCapacity,
                    Technology,Region]]

                    df_AggregateGen3=pd.concat([df_AggregateGen3,pd.DataFrame(Data,columns=df_AggregateGen3.columns)],ignore_index=True)

In [109]:
df_AggregateGen3.shape

(166, 10)

In [110]:
df_AggregateGen3.to_excel("test.xlsx")